In [19]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import numpy as np

In [20]:
#
PROBLEM_SIZE = 5
NUM_SETS = 10
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

print(SETS)

(array([ True,  True, False, False, False]), array([False, False,  True, False, False]), array([False, False, False,  True, False]), array([ True, False, False, False, False]), array([False, False,  True, False, False]), array([ True, False,  True,  True,  True]), array([ True,  True, False, False, False]), array([False,  True, False, False, False]), array([False, False,  True,  True,  True]), array([False,  True,  True, False, False]))


In [21]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [22]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [23]:
def g(state):
    return len(state.taken)

In [24]:
def h0(state):
    # Define a heuristic function that estimates the remaining cost to reach the goal
    return PROBLEM_SIZE - sum(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))


def covered(state):
    # Calculate the elements that have been covered by the selected sets
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.zeros(PROBLEM_SIZE, dtype=bool),  # Use np.zeros to create a boolean array
    )

def sum_second_element(x):
    return np.sum(x[1])

def calculate_frequency(i, already_covered, state):
    set_ = SETS[i]
    # Calculate the overlap with the already covered elements
    overlap = np.sum(np.logical_and(set_, already_covered))
    # Calculate the potential coverage of this set (number of uncovered elements it can cover)
    potential_coverage = np.sum(np.logical_and(set_, np.logical_not(already_covered)))
    # Calculate the frequency as the ratio of potential coverage to (overlap + 1) to avoid division by zero
    frequency = potential_coverage / (overlap + 1)
    return frequency

def h(state):
    already_covered = covered(state)
    if np.all(already_covered):
        # If all elements are covered, no need to select more sets
        return 0
    
    missing_size = PROBLEM_SIZE - np.sum(already_covered)  # Use np.sum for summing boolean array

    frequencies = [(i, calculate_frequency(i, already_covered, state)) for i in state.not_taken]
    sorted_frequencies = sorted(frequencies, key=sum_second_element, reverse=True)
    candidates = [SETS[i] for i, _ in sorted_frequencies]

    taken = 0  # Initialize to 0 since no sets have been taken yet
    covered_elements = np.zeros(PROBLEM_SIZE, dtype=bool)

    while np.sum(covered_elements) < missing_size:
        covered_elements |= candidates[taken]
        taken += 1

    return taken


In [25]:
def h_g(state):
    return  h(state)+g(state)

In [26]:
# Create a priority queue for A* search
frontier = PriorityQueue()

# Initialize the initial state with no sets taken and all sets not taken
initial_state = State(set(), set(range(NUM_SETS)))

# Push the initial state into the priority queue with the heuristic value as priority
frontier.put((h_g(initial_state), initial_state))

# Initialize a counter to count the number of steps
counter = 0

# Get the state with the highest priority from the priority queue
_, current_state = frontier.get()

# Loop until the goal state is reached
while not goal_check(current_state):
    counter += 1

    # Iterate through possible actions (taking sets)
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put((h_g(new_state), new_state))

    # Get the state with the highest priority from the priority queue for the next iteration
    _, current_state = frontier.get()

# Print the number of steps required to solve the problem
print(f"Solved in {counter} steps")

# Print the heuristic value of the final state
print("Function:", h_g(current_state))

# Return the final state
current_state


Solved in 4 steps
Function: 2


State(taken={0, 5}, not_taken={1, 2, 3, 4, 6, 7, 8, 9})